# Practical Work in AI: Enhancing item relevance scores with psychology-based interest over time for music recommender systems

Author: Laura Legat
Matriculation ID: 51868012

In [ ]:
# import access to Google Drive files

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import needed libraries

import os
import torch
import sys
import pandas as pd
import importlib

In [ ]:
# create train-val-test sets for Ex2Vec training, as well as sequences for GRU4Rec training
!python /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/preprocess.py

In [ ]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
# import custom code

# Append the directory containing 'data_sampler' and 'ex2vec' to Python's search path
sys.path.append('/content/drive/MyDrive/JKU/practical_work/Practical-Work-AI')

# imports modules for preparing data and for training/evaluating the ex2vec model
import data_sampler
from ex2vec import Ex2VecEngine

#import dir of gru4rec_pytorch module to python path in order to be able to access GRU4Rec model class to be able to load it
sys.path.append('/content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/GRU4Rec_PyTorch_Fork')
from gru4rec_pytorch
import evaluation as GRUeval

The size of the training set is: 1106989
The size of the validation set is: 156748
The size of the test set is: 320078


In [ ]:
# Check if gpu is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_name = torch.cuda.get_device_name(device)
    print(f'Using GPU: {device_name}')
else:
    device = torch.device('cpu')
    print('Using CPU')

print(f'Current device: {device}')

Using GPU: Tesla T4
Current device: cuda


In [ ]:
# check that deezer parameter file exists
param_file_path = '/content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/GRU4Rec_PyTorch_Fork/paramfiles/deezer_paramfile.py'
assert os.path.isfile(param_file_path), f'Parameter file not found at {param_file_path}'

In [ ]:
# train baseline Ex2Vec with Ex2Vec item embeddings
!python /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/train.py

In [ ]:
# train GRU4Rec

"""
params: (from https://github.com/hidasib/GRU4Rec/blob/master/README.md)
  -t    Testset path
  -pf   Parameter file path
  -s    Path to save the state dict to
  -m    Calculate recall, MRR etc. at the given list length
  -ik   Item key
  -tk   Timestamp key
  -d    Device
"""

!python /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/GRU4Rec_PyTorch_Fork/run.py /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/data/seq_train.csv -t /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/data/seq_val.csv -pf /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/GRU4Rec_PyTorch_Fork/paramfiles/deezer_paramfile.py -s /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/models/GRU4Rec.pt -m 1 5 10 20 -ik "itemId" -tk "timestamp"

In [ ]:
# re-train Ex2Vec with GRU4Rec item embeddings
!python /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/train.py -ep /content/drive/MyDrive/JKU/practical_work/Practical-Work-AI/models/GRU4Rec.pt

The size of the training set is: 1106989
The size of the validation set is: 156748
The size of the test set is: 320078
Ex2Vec(
  (user_lamb): Embedding(3623, 1)
  (user_bias): Embedding(3623, 1)
  (item_bias): Embedding(879, 1)
  (embedding_user): Embedding(3623, 64)
  (embedding_item): Embedding(879, 64)
  (logistic): Sigmoid()
)
global_lamb <class 'torch.Tensor'> torch.Size([])
alpha <class 'torch.Tensor'> torch.Size([])
beta <class 'torch.Tensor'> torch.Size([])
gamma <class 'torch.Tensor'> torch.Size([])
cutoff <class 'torch.Tensor'> torch.Size([])
user_lamb.weight <class 'torch.Tensor'> torch.Size([3623, 1])
user_bias.weight <class 'torch.Tensor'> torch.Size([3623, 1])
item_bias.weight <class 'torch.Tensor'> torch.Size([879, 1])
embedding_user.weight <class 'torch.Tensor'> torch.Size([3623, 64])
embedding_item.weight <class 'torch.Tensor'> torch.Size([879, 64])
Using validation set for evaluation

started training model:  ex2vec_BS512LR5e-05L_DIM64
Epoch 0 starts !
100% 2163/2163 

In [ ]:
# retrain Ex2Vec with GRU4Rec item embeddings

In [ ]:
# compare performance